In [2]:
#以下部分定义一个决策树模型实现对winequality-red.csv数据集的分类
import pandas as pd
data = pd.read_csv('winequality-red.csv',sep=';')
import numpy as np
data = np.array(data)
print(data)

[[ 7.4    0.7    0.    ...  0.56   9.4    5.   ]
 [ 7.8    0.88   0.    ...  0.68   9.8    5.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    5.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     6.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    5.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     6.   ]]


In [3]:
#Count the sample size for different categories by the last column
count = np.array([0]*10)
for i in range(len(data)):
    count[int(data[i][-1])] = count[int(data[i][-1])] + 1
#打印每个类别的样本数量->仅仅针对较多的类别进行预测
print(count)

[  0   0   0  10  53 681 638 199  18   0]


In [4]:
#划分数据集为1:4,使用numpy的random.shuffle()函数
np.random.shuffle(data)
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]
print(train_data,len(train_data))
print(test_data,len(test_data))
n = train_data.shape[1]
print(n)

[[ 7.6   0.95  0.03 ...  0.56  9.6   5.  ]
 [ 6.7   0.54  0.13 ...  0.64  9.8   5.  ]
 [ 6.8   0.81  0.05 ...  0.66 10.8   6.  ]
 ...
 [ 6.1   0.32  0.25 ...  0.97 10.6   5.  ]
 [ 8.2   0.34  0.37 ...  0.81 12.    6.  ]
 [ 7.2   1.    0.   ...  0.46 10.    5.  ]] 1279
[[ 5.4    0.74   0.09  ...  0.56  11.6    6.   ]
 [11.4    0.6    0.49  ...  0.63  10.5    6.   ]
 [ 8.6    0.685  0.1   ...  0.65   9.55   6.   ]
 ...
 [ 8.     0.83   0.27  ...  0.48   9.8    4.   ]
 [ 7.2    0.53   0.14  ...  0.61  12.1    6.   ]
 [10.4    0.55   0.23  ...  0.64  10.3    6.   ]] 320
12


In [5]:
#定义feature项为前8列，label项为最后一列
train_feature = train_data[:,:-1]
#再次切分出train_feature的前8列为train_feature1
train_label = train_data[:,-1]
test_feature = test_data[:,:-1]
test_label = test_data[:,-1]
#print(train_feature)
#获取列数
n = train_feature.shape[1]
print(n)

11


In [ ]:
#给前8列添加一个标签，用于后续的决策树的构建
feature_label = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density']

In [8]:
#定义一个新的numpy数组,将train_label转换为one-hot编码,打印数据的形状
train_label1 = np.zeros((len(train_label),10))
for i in range(len(train_label)):
    train_label1[i][int(train_label[i])] = 1
print(train_label1)
print(train_label1.shape)
#定义一个新的numpy数组,将test_label转换为one-hot编码
test_label1 = np.zeros((len(test_label),10))
for i in range(len(test_label)):
    test_label1[i][int(test_label[i])] = 1
print(test_label1)
print(test_label1.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1279, 10)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(320, 10)


In [9]:
#取出train_label1中的第一列
print(train_label1[:,0])
#返回train_label1中的第一列的形状
print(train_label1[:,0].shape)

[0. 0. 0. ... 0. 0. 0.]
(1279,)


In [15]:

#将训练集和测试集标准化
train_feature = (train_feature - np.mean(train_feature,axis=0))/np.std(train_feature,axis=0)
test_feature = (test_feature - np.mean(test_feature,axis=0))/np.std(test_feature,axis=0)
print(train_feature)
print(test_feature)
#打印数据的形状
print(train_feature.shape)
print(test_feature.shape)

[[-0.41965847  2.36949956 -1.24495856 ... -0.72639127 -0.57948219
  -0.77508878]
 [-0.93858517  0.06172234 -0.72826258 ...  1.98453547 -0.10453331
  -0.58597121]
 [-0.88092664  1.58147807 -1.14161936 ...  1.32333383  0.0142039
   0.35961665]
 ...
 [-1.2845363  -1.17659715 -0.10822741 ...  0.72825235  1.85463078
   0.17049908]
 [-0.07370734 -1.06402265  0.51180777 ... -0.52803077  0.90473304
   1.49432208]
 [-0.65029256  2.65093581 -1.39996735 ...  0.79437251 -1.17316828
  -0.39685364]]
[[-1.63585883  1.17826167 -0.90785083 ...  2.13111561 -0.57839077
   1.06208265]
 [ 1.76406426  0.4182023   1.09741071 ... -1.00044248 -0.17477852
   0.05827053]
 [ 0.17743349  0.87966692 -0.85771929 ... -0.03688614 -0.05946073
  -0.80865811]
 ...
 [-0.16255882  1.66687125 -0.00548314 ... -0.15733068 -1.03966192
  -0.580519  ]
 [-0.6158819   0.03817262 -0.65719314 ...  0.20400294 -0.29009631
   1.51836089]
 [ 1.19741042  0.14675253 -0.20600929 ... -0.57888658 -0.11711963
  -0.12424076]]
(1279, 11)
(320, 